In [ ]:
import xarray as xr

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import os

In [ ]:
error_paths = {
    127:'/home/lucas/h2ox-ai/experiments_2/sacred/127/errors.nc',
    140:'/home/lucas/h2ox-ai/experiments_2/sacred/140/errors.nc',
    141:'/home/lucas/h2ox-ai/experiments_2/sacred/141/errors.nc',
    142:'/home/lucas/h2ox-ai/experiments_2/sacred/142/errors.nc',
    144:'/home/lucas/h2ox-ai/experiments_2/sacred/144/errors.nc',
}

In [ ]:
dss = {kk:xr.open_dataset(vv) for kk,vv in error_paths.items()}

In [ ]:
for kk in dss.keys():
    print (kk)
    dss[kk]['pearson-r'].mean(dim=('site')).plot()
    plt.show()

In [ ]:
import pickle

In [ ]:
var_norms = pickle.load(open('./../models/kaveri_preprocessing.pkl','rb'))

In [ ]:
std_target = dict(
            zip(
                var_norms["std_norm"]["shift_targets_WATER_VOLUME"]["std"].to_dict()[
                    "coords"
                ]["global_sites"]["data"],
                var_norms["std_norm"]["shift_targets_WATER_VOLUME"]["std"].to_dict()[
                    "data"
                ],
            )
        )

In [ ]:
import torch

In [ ]:
std_target

### prep a data sample

In [ ]:
import xarray as xr

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import json, yaml

In [ ]:
sites = json.load(open('./../data/kaveri.json','r'))

In [ ]:
cfg = yaml.load(open('./../conf-all.yaml','r'), Loader=yaml.SafeLoader)

In [ ]:
ds = xr.open_dataset('./../data/data_kaveri.nc')

In [ ]:
ds

In [ ]:
{'<pred_date>':{'<site>':{'historic':{'<var>':},'forecast':{'<var>':},'future':{'<var>':}}}}

In [ ]:
target_date = datetime(2020,10,1)
ini_date = target_date - timedelta(days=91)

In [ ]:
hist = ds[[var for var in cfg['dataset_parameters']['historic_variables'] if var not in cfg['dataset_parameters']['target_var']]].sel({'global_sites':sites,'date':pd.date_range(ini_date, target_date, freq='1D'),'steps':timedelta(days=0)})

In [ ]:
target = ds[cfg['dataset_parameters']['target_var']].sel({'global_sites':sites,'date':pd.date_range(ini_date, target_date, freq='1D'),'steps':timedelta(days=0)})

In [ ]:
forecast = ds[[var for var in cfg['dataset_parameters']['forecast_variables']]].sel({'global_sites':sites,'date':target_date, 'steps':[timedelta(days=ii) for ii in range(0,15)]})

In [ ]:
future = ds[[var for var in cfg['dataset_parameters']['future_variables']]].sel({'global_sites':sites,'date':target_date, 'steps':[timedelta(days=ii) for ii in range(15,91)]})

In [ ]:
hist_dict = hist.to_array().transpose('global_sites','variable','date').to_dict()

In [ ]:
target_dict = target.to_array().transpose('global_sites','variable','date').to_dict()

In [ ]:
forecast_dict = forecast.to_array().transpose('global_sites','variable','steps').to_dict()

In [ ]:
future_dict = future.to_array().transpose('global_sites','variable','steps').to_dict()

In [ ]:
def reshape_data(data_dict):
    """{'<site>':{'<var>':<data>}}"""
    res_dict = {}
    for ii_s, site in enumerate(data_dict['coords']['global_sites']['data']):
        res_dict[site] = dict(zip(data_dict['coords']['variable']['data'],data_dict['data'][ii_s]))
        
    return res_dict

In [ ]:
def combine_data(hist, target, forecast, future):
    combined_data = {}
    for site in hist.keys():
        combined_data[site] = {
            'x_d':hist[site],
            'x_f':forecast[site],
            'x_ff':future[site],
            'y':target[site],
        }
    return combined_data

In [ ]:
hist_dict = reshape_data(hist_dict)
forecast_dict = reshape_data(forecast_dict)
future_dict = reshape_data(future_dict)
target_dict = reshape_data(target_dict)

In [ ]:
combined_dict = combine_data(hist_dict, target_dict, forecast_dict, future_dict)

In [ ]:
sample_data = {target_date.isoformat()[0:10]:combined_dict}

In [ ]:
json.dump(sample_data,open('./../data/kaveri_sample_2020_10_01.json','w'))

In [ ]:
tt = torch.rand(2,5,2)

In [ ]:
tt.permute(2,0,1).shape

In [ ]:
var_norms_json

In [ ]:
#### check against a sample
tt_data = pickle.load(open('./../data/ds_data.pkl','rb'))

In [ ]:
tt_data[0]['narayanapura']['x_ff'].shape

In [ ]:
stack = torch.stack([torch.rand(10,3) for _ in range(5)])#.sort(dim=0).values.cumsum(dim=1)

In [ ]:
stack.sort(dim=0).values.cumsum(dim=1).shape

In [ ]:
hist_dict['coords']['global_sites']['data']

In [ ]:
hist_dict.keys()

In [ ]:
hist.to_dict()

In [ ]:
ds = xr.open_dataset(pp)

In [ ]:
ds['pearson-r'].mean(dim=('site'))